# Programming Assignment 3: Information Retrieval

In this assignment you will be improving upon a rather poorly-made information retrieval system. You will build an inverted index to quickly retrieve documents that match queries and then make it even better by using term-frequency inverse-document-frequency weighting and cosine similarity to compare queries to your data set. Your IR system will be evaluated for accuracy on the correct documents retrieved for different queries and the correctly computed tf-idf values and cosine similarities.

## Data

You will be using your IR system to find relevant documents among a collection of sixty short stories by Rider Haggard. The training data is located in the data/ directory under the subdirectory RiderHaggard/. Within this directory you will see yet another directory raw/. This contains the raw text files of the sixty short stories. The data/ directory also contains the files dev_queries.txt and dev_solutions.txt. We have provided these to you as a set of development queries and their expected answers to use as you begin implementing your IR system.

## Your Task

Improve upon the given IR system by implementing the following features:

<b>Inverted Positional Index:</b> Implement an inverted index - a mapping from words to the documents in which they occur, as well as the positions in the documents for which they occur.

<b>Boolean Retrieval:</b> Implement a Boolean retrieval system, in which you return the list of documents that contain all words in a query. (Yes, you only need to support conjunctions for this assignment.)

<b>Phrase Query Retrieval:</b> Implement a system that returns the list of documents in which the full phrase appears, (ie. the words of the query appear next to each other, in the specified order). Note that at the time of retrieval, you will not have access to the original documents anymore (the documents would be turned into bag of words), so you'll have to utilize your inverted positional index to complete this part.

<b>TF-IDF:</b> Compute and store the term-frequency inverse-document-frequency value for every word-document co-occurrence:

<b>Cosine Similarity:</b> Implement cosine similarity in order to improve upon the ranked retrieval system, which currently retrieves documents based upon the Jaccard coefficient between the query and each document. Also note that when computing $w_{t, q}$ (i.e. the weight for the word 𝑤 in the query) do not include the idf term. That is, $w_{t, q} = 1 + \log_{10} \text{tf}_{t, q}$.
<b> The reference solution uses ltc.lnn weighting for computing cosine scores. </b>

## Evaluation

Your IR system will be evaluated on a development set of queries as well as a held-out set of queries. The queries are encoded in the file <b>dev_queries.txt</b> and are:

- separation of church and state
- white-robed priests
- ancient underground city
- native african queen
- zulu king

We test your system based on the five parts mentioned above: the inverted index (used both to get word positions and to get postings), boolean retrieval, phrase query retrieval, computing the correct tf-idf values, and implementing cosine similarity using the tf-idf values.

### Environment Check

Before we do anything else, let's quickly check that you're running the correct
version of Python and are in the right environment!

In [1]:
import os
assert os.environ['CONDA_DEFAULT_ENV'] == "cs124"

import sys
assert sys.version_info.major == 3 and sys.version_info.minor == 8

If the above cell complains, it means that you're using the wrong environment
or Python version!

If so, please exit this notebook, kill the notebook server with CTRL-C, and
try running

$ conda activate cs124

then restarting your notebook server with

$ jupyter notebook

If that doesn't work, you should go back and follow the installation
instructions in PA0.

## Getting Started!

We will first start by importing some modules and setting up our IR system class.


In [2]:
import json
import os
import re
import sys
import math

from porter_stemmer import PorterStemmer

In [3]:
class IRSystem:
    def __init__(self):
        # For holding the data - initialized in read_data()
        self.titles = []
        self.docs = []
        self.vocab = []
        # For the text pre-processing.
        self.alphanum = re.compile('[^a-zA-Z0-9]')
        self.p = PorterStemmer()

    def get_uniq_words(self):
        uniq = set()
        for doc in self.docs:
            for word in doc:
                uniq.add(word)
        return uniq

    def __read_raw_data(self, dirname):
        print("Stemming Documents...")

        titles = []
        docs = []
        os.mkdir('%s/stemmed' % dirname)
        title_pattern = re.compile('(.*) \d+\.txt')
        # for debugging 
        #title_pattern = re.compile('(.*).txt')

        # make sure we're only getting the files we actually want
        filenames = []
        for filename in os.listdir('%s/raw' % dirname):
            if filename.endswith(".txt") and not filename.startswith("."):
                filenames.append(filename)
                

        for i, filename in enumerate(filenames):
            title = title_pattern.search(filename).group(1)
            print("    Doc %d of %d: %s" % (i + 1, len(filenames), title))
            titles.append(title)
            contents = []
            f = open('%s/raw/%s' % (dirname, filename), 'r', encoding="utf-8")
            of = open('%s/stemmed/%s.txt' % (dirname, title), 'w',
                      encoding="utf-8")
            for line in f:
                # make sure everything is lower case
                line = line.lower()
                # split on whitespace
                line = [xx.strip() for xx in line.split()]
                # remove non alphanumeric characters
                line = [self.alphanum.sub('', xx) for xx in line]
                # remove any words that are now empty
                line = [xx for xx in line if xx != '']
                # stem words
                line = [self.p.stem(xx) for xx in line]
                # add to the document's conents
                contents.extend(line)
                if len(line) > 0:
                    of.write(" ".join(line))
                    of.write('\n')
            f.close()
            of.close()
            docs.append(contents)
        return titles, docs

    def __read_stemmed_data(self, dirname):
        print("Already stemmed!")
        titles = []
        docs = []

        # make sure we're only getting the files we actually want
        filenames = []
        for filename in os.listdir('%s/stemmed' % dirname):
            if filename.endswith(".txt") and not filename.startswith("."):
                filenames.append(filename)

        
        #comment for debugging purposes
        if len(filenames) != 60:
            msg = "There are not 60 documents in ../data/RiderHaggard/stemmed/\n"
            msg += "Remove ../data/RiderHaggard/stemmed/ directory and re-run."
            raise Exception(msg)
        

        for i, filename in enumerate(filenames):
            title = filename.split('.')[0]
            titles.append(title)
            contents = []
            f = open('%s/stemmed/%s' % (dirname, filename), 'r',
                     encoding="utf-8")
            for line in f:
                # split on whitespace
                line = [xx.strip() for xx in line.split()]
                # add to the document's conents
                contents.extend(line)
            f.close()
            docs.append(contents)
            
        

        return titles, docs

    def read_data(self, dirname):
        """
        Given the location of the 'data' directory, reads in the documents to
        be indexed.
        """
        # NOTE: We cache stemmed documents for speed
        #       (i.e. write to files in new 'stemmed/' dir).

        print("Reading in documents...")
        # dict mapping file names to list of "words" (tokens)
        filenames = os.listdir(dirname)
        subdirs = os.listdir(dirname)
        if 'stemmed' in subdirs:
            titles, docs = self.__read_stemmed_data(dirname)
        else:
            titles, docs = self.__read_raw_data(dirname)

        # Sort document alphabetically by title to ensure we have the proper
        # document indices when referring to them.
        ordering = [idx for idx, title in sorted(enumerate(titles),
                                                 key=lambda xx: xx[1])]

        self.titles = []
        self.docs = []
        numdocs = len(docs)
        for d in range(numdocs):
            self.titles.append(titles[ordering[d]])
            self.docs.append(docs[ordering[d]])

        # Get the vocabulary.
        self.vocab = [xx for xx in self.get_uniq_words()]

In [4]:
#testing the inverted positional index logic 

document_1 = ['I', 'love', 'Mona']
document_2 = ['Mona', 'is', 'so', 'cute']
document_3 = ['Oh', 'Mona', 'Mona', 'so', 'cute']

documents = [document_1, document_2, document_3]

inv_index_test = {}

for doc_id_test in range(len(documents)): 
    for position_test in range(len(documents[doc_id_test])): 
        word_test = documents[doc_id_test][position_test]
        if word_test in inv_index_test.keys():
            if doc_id_test in inv_index_test[word_test].keys():
                inv_index_test[word_test][doc_id_test].append(position_test)
            else: 
                inv_index_test[word_test][doc_id_test] = [position_test]
        else: 
            inv_index_test[word_test] = {doc_id_test: [position_test]}
            
            

In [20]:
document_1


['I', 'love', 'Mona']

In [21]:
inv_index_test


{'I': {0: [0]},
 'love': {0: [1]},
 'Mona': {0: [2], 1: [0], 2: [1, 2]},
 'is': {1: [1]},
 'so': {1: [2], 2: [3]},
 'cute': {1: [3], 2: [4]},
 'Oh': {2: [0]}}

In [22]:
#turn documents into a map from ID to bag of words 
map_id_to_words = {}
for d, doc in enumerate(documents): 
    bag_word = set(doc)
    map_id_to_words[d] = bag_word 
    
documents = map_id_to_words


You will first build the inverted positional index (data structure that keeps track of the documents in which a particular word is contained, and the positions of that word in the document). The documents will have already been read in at this point. The following instance variables in the class are included in the starter code for you to use to build your inverted positional index: titles (a list of strings), docs (a list of lists of strings), and vocab (a list of strings).

print(documents)

In [23]:
documents

{0: {'I', 'Mona', 'love'},
 1: {'Mona', 'cute', 'is', 'so'},
 2: {'Mona', 'Oh', 'cute', 'so'}}

In [24]:
doc1_ID = list(documents.keys())[list(documents.values()).index(set(document_1))]

In [25]:
doc1_ID


0

In [14]:
inv_index_test['Mona'][doc1_ID]

[2]

In [17]:
list(inv_index_test['Mona'].keys())

[0, 1, 2]

In [23]:
query = ['Mona', 'Oh', 'cute']

postings = []
len_postings = []
for word in query: 
    posting = list(inv_index_test[word].keys())
    postings.append(posting)
    len_postings.append(len(posting))
    

In [24]:
postings

[[0, 1, 2], [2], [1, 2]]

In [25]:
len_postings

[3, 1, 2]

In [26]:
shorter_posting_index = len_postings.index(min(len_postings))

In [27]:
shorter_posting_index

1

In [28]:
shorter_posting = postings[shorter_posting_index]
shorter_posting

[2]

In [30]:
del postings[shorter_posting_index]
postings

[[0, 1, 2]]

In [4]:
class IRSystem(IRSystem):
    def index(self):
        """
        Build an index of the documents.
        """
        print("Indexing...")
        # ------------------------------------------------------------------
        # TODO: Create an inverted, positional index.
        #       Granted this may not be a linked list as in a proper
        #       implementation.
        #       This index should allow easy access to both 
        #       1) the documents in which a particular word is contained, and 
        #       2) for every document, the positions of that word in the document 
        #       Some helpful instance variables:
        #         * self.docs = List of documents
        #         * self.titles = List of titles   => Why do we need self.titles ? 

        inv_index = {}
        
        '''
        For inverted index : 
        For each term in each doc, get (term, docID) pairs. 
        Order the pairs alphabetically. 
        Merge in order to have (term, [docIDs])
        For inverted positional index: 
        For each term, we need postings of the form docID: <position1, position2, ...> for all docIDs containing the term. 
        
        inv_index = { term : {docID: [position1, position2...], docID: [position1, ...], ...}, term: dict of docs -> positions...} 
        
        docID will be the index of the document in self.docs 
        
        The inverted index will be a dictionnary, where the keys are the terms of the vocab, and the values are a dictionnary (where the keys are the docID of each doc containing the term, and the values is a list of the positions in the doc). 
        
        '''

        # Generate inverted index here
        for doc_id in range(len(self.docs)): #for each doc, the doc_id is its position in the self.docs list 
            for position in range(len(self.docs[doc_id])): # for each word in the doc 
                word = self.docs[doc_id][position]
                if word in inv_index.keys(): # if the word already exists in the dictionary 
                    #append its position to the list of positions inside the dict. 
                    if doc_id in inv_index[word].keys(): 
                        inv_index[word][doc_id].append(position)
                    else: 
                        inv_index[word][doc_id] = [position]
                else: 
                    inv_index[word] = {doc_id: [position]}

        self.inv_index = inv_index

        # ------------------------------------------------------------------

        # turn self.docs into a map from ID to bag of words
        id_to_bag_of_words = {}
        for d, doc in enumerate(self.docs):
            bag_of_words = set(doc)
            id_to_bag_of_words[d] = bag_of_words
        self.docs = id_to_bag_of_words
        


In [7]:
from collections import defaultdict 

test_dict = defaultdict(dict)



In [8]:
test_dict = defaultdict(lambda: defaultdict(list))

In [9]:
test_dict_2 = defaultdict(lambda: [])

In [11]:
test_dict_2[3].append(1)

In [12]:
test_dict_2

defaultdict(<function __main__.<lambda>()>, {3: [1]})

In [13]:
test_dict[2]

defaultdict(list, {})

In [14]:
test_dict['hello'][3]

[]

Next we will implement get_word_positions. This method returns a list of integers that identifies the positions in the document doc in which the word is found.  This is basically just an API into your inverted index, but you must implement it in order for the index to be evaluated fully.

In [5]:
class IRSystem(IRSystem):
    def get_word_positions(self, word, doc):
        """
        Given a word and a document, use the inverted index to return
        
        the positions of the specified word in the specified document.
        """
        # ------------------------------------------------------------------
        # TODO: return the list of positions for a word in a document.
        positions = []
               
        #get the docID from self.docs
        #docID = list(self.docs.keys())[list(self.docs.values()).index(set(doc))]
        #for testing purposes 

        
        # return inv_index[word][docID]
        positions = self.inv_index[word][doc]

        return positions
        # ------------------------------------------------------------------

We will add another method, get_posting, that returns a list of integers (document IDs) that identifies the documents in which the word is found. This is basically another API into your inverted index, but you must implement it in order to be evaluated fully.

Keep in mind that the document IDs in each postings list to be sorted in order to perform the linear merge for boolean retrieval.

In [6]:
class IRSystem(IRSystem):
    def get_posting(self, word):
        """
        Given a word, this returns the list of document indices (sorted) in
        which the word occurs.
        """
        # ------------------------------------------------------------------
        # TODO: return the list of postings for a word.
        posting = []
        
        posting = list(self.inv_index[word].keys())

        return posting
        # ------------------------------------------------------------------
    
    def get_posting_unstemmed(self, word):
        """
        Given a word, this *stems* the word and then calls get_posting on the
        stemmed word to get its postings list. You should *not* need to change
        this function. It is needed for submission.
        """
        word = self.p.stem(word)
        return self.get_posting(word)

Next, we will implement Boolean retrieval, returning a list of document IDs corresponding to the documents in which all the words in query occur.

Please implement the linear merge algorithm outlined in the videos/book (do not use built-in set intersection functions).

In [7]:
class IRSystem(IRSystem):
    
    def intersect(self, postings_1, postings_2):
        i = 0
        j = 0 
        intersect = []
        while i != len(postings_1) and j != len(postings_2):
            if postings_1[i] == postings_2[j]:
                intersect.append(postings_1[i])
                i += 1
                j += 1
            else: 
                if postings_1[i] < postings_2[j]:
                    i += 1
                else: 
                    j += 1 
        return intersect 
        
    
    def boolean_retrieve(self, query):
        """
        Given a query in the form of a list of *stemmed* words, this returns
        the list of documents in which *all* of those words occur (ie an AND
        query).
        Return an empty list if the query does not return any documents.
        """
        # ------------------------------------------------------------------
        # TODO: Implement Boolean retrieval. You will want to use your
        #       inverted index that you created in index().
        # Right now this just returns all the possible documents!
        docs = []
        postings = []
        postings_length = []
        
        # first for each word in the query, get posting unstemmed 
        for word in query: 
            posting = self.get_posting_unstemmed(word)
            postings.append(posting)
            postings_length.append(len(posting))
        
        # get the intersection of all the postings in order of increasing document frequency (i.e. length of the postings)
        while len(postings) > 1: 
            # get the shorter posting : create a helper function  
            shorter_posting_index = postings_length.index(min(postings_length))
            shorter_posting = postings[shorter_posting_index]
            del postings_length[shorter_posting_index]
            del postings[shorter_posting_index]
            
            #get the second shortest : same helper function probably 
            shorter_posting_index2 = postings_length.index(min(postings_length))
            shorter_posting2 = postings[shorter_posting_index2]
            del postings_length[shorter_posting_index2]
            del postings[shorter_posting_index2]
            
            intersect = self.intersect(shorter_posting, shorter_posting2)
            
            # reinsert the intersect into the postings 
            if len(intersect) != 0: 
                postings.append(intersect)
                postings_length.append(len(intersect))
                
            else: 
                break 
                
        # the if is optionnal here right ? 
        if len(postings) == 1: 
            docs = postings[0]
            
        
        
        # ------------------------------------------------------------------

        return sorted(docs)  # sorted doesn't actually matter

Let's continue to phase query retrieval. Our phrase_retrieve method will return a list of document IDs corresponding to the documents in which all the actual query phrase occurs.

In [14]:
class IRSystem(IRSystem):
    
    def intersect_k(self, positions_1, positions_2, k): 
        pos_1 = 0
        pos_2 = 0
        intersect = []
        while pos_1 != len(positions_1) and pos_2 != len(positions_2): 
            if positions_2[pos_2] - positions_1[pos_1] == k:
                intersect.append(positions_1[pos_1])
                pos_1 += 1
                pos_2 += 1
            else: 
                if positions_2[pos_2] < positions_1[pos_1]:
                    pos_2 += 1
                else: 
                    pos_1 += 1
        return intersect 
    
    def phrase_retrieve(self, query):
        """
        Given a query in the form of an ordered list of *stemmed* words, this 
        returns the list of documents in which *all* of those words occur, and 
        in the specified order. 
        Return an empty list if the query does not return any documents. 
        """
        # ------------------------------------------------------------------
        # TODO: Implement Phrase Query retrieval (ie. return the documents 
        #       that don't just contain the words, but contain them in the 
        #       correct order) You will want to use the inverted index 
        #       that you created in index(), and may also consider using
        #       boolean_retrieve. 
        #       NOTE that you no longer have access to the original documents
        #       in self.docs because it is now a map from doc IDs to set
        #       of unique words in the original document.
        # Right now this just returns all possible documents!
        docs = []
        boolean_docs = self.boolean_retrieve(query)
        result = []
        
        if len(query) <= 1: 
            docs = boolean_docs 
            
        else: 
            for doc in boolean_docs: 
                positions = []
                for word in query: 
                    positions.append(self.get_word_positions(word, doc))
                #query has at least two words 
                result = self.intersect_k(positions[0], positions[1], 1)
                i = 2
                while i != len(query) and result != []:
                    result = self.intersect_k(result, positions[i], i)
                    i += 1
                if result != []:
                    docs.append(doc)
                              
        # ------------------------------------------------------------------

        return sorted(docs)  # sorted doesn't actually matter

In [ ]:
docs = [['I', 'love', 'Mona', 'a', 'lot', 'I'], ['everybody', 'love', 'Mona'], ['Mona', 'and', 'I', 'love']]
query = ['I', 'love', 'Mona']
result = {}
postings = [{0: [0], 2: [2]}, {0: [1], 1: [1]}, {0: [2], 1: [2], 2: [0]}]



In [30]:
def intersect_k_test(positions_1, positions_2, k): 
        pos_1 = 0
        pos_2 = 0
        intersect = []
        while pos_1 != len(positions_1) and pos_2 != len(positions_2): 
            if positions_2[pos_2] - positions_1[pos_1] == k:
                intersect.append(positions_1[pos_1])
                pos_1 += 1
                pos_2 += 1
            else: 
                if positions_2[pos_2] < positions_1[pos_1]:
                    pos_2 += 1
                else: 
                    pos_1 += 1
        return intersect 

In [33]:

positions_test = [[0,5], [1], [2]]

result_test = intersect_k_test(positions_test[0], positions_test[1], 1)

result_test = intersect_k_test(result_test, positions_test[2], 2)


In [34]:
result_test

[0]

Now we will compute and score the td-idf values. compute_tfidf and stores the tf-idf values for words and documents. For this you will probably want to be aware of the class variable vocab, which holds the list of all unique words, as well as the inverted index you created earlier.

You must also implement get_tfidf to return the tf-idf weight for a particular word and document ID.

In [15]:
class IRSystem(IRSystem):
    def compute_tfidf(self):
        # -------------------------------------------------------------------
        # TODO: Compute and store TF-IDF values for words and documents in self.tfidf.
        #       Recall that you can make use of:
        #         * self.vocab: a list of all distinct (stemmed) words
        #         * self.docs: a list of lists, where the i-th document is
        #                   self.docs[i] => ['word1', 'word2', ..., 'wordN']
        #       NOTE that you probably do *not* want to store a value for every
        #       word-document pair, but rather just for those pairs where a
        #       word actually occurs in the document.
        print("Calculating tf-idf...")
        self.tfidf = {}
        

        # ------------------------------------------------------------------

    def get_tfidf(self, word, document):
        # ------------------------------------------------------------------
        # TODO: Return the tf-idf weigthing for the given word (string) and
        #       document index.
        tfidf = 0.0
        # ------------------------------------------------------------------
        return tfidf

    def get_tfidf_unstemmed(self, word, document):
        """
        This function gets the TF-IDF of an *unstemmed* word in a document.
        Stems the word and then calls get_tfidf. You should *not* need to
        change this interface, but it is necessary for submission.
        """
        word = self.p.stem(word)
        return self.get_tfidf(word, document)

Lastly, we will implement rank_retrieve. This function returns a priority queue of the top ranked documents for a given query. Right now it ranks documents according to their Jaccard similarity with the query, but you will replace this method of ranking with a ranking using the <b>cosine similarity</b> between the documents and query.
    
Remember to use ltc.lnn weighting! This means that the query vector weights will be $1 + \log_{10} \text{tf}_{t, q}$ with no IDF term or normalization, and we only normalize by the length of the document vector (square root of the sum of squares of the tf-idf weights).
    
When we say normalize by "document length" or "length of document", we mean the length of the document vector, NOT the number of words in the actual text document.

In [16]:
class IRSystem(IRSystem):
    def rank_retrieve(self, query):
        """
        Given a query (a list of words), return a rank-ordered list of
        documents (by ID) and score for the query.
        """
        scores = [0.0 for xx in range(len(self.titles))]
        # ------------------------------------------------------------------
        # TODO: Implement cosine similarity between a document and a list of
        #       query words.

        # Right now, this code simply gets the score by taking the Jaccard
        # similarity between the query and every document.
        words_in_query = set()
        for word in query:
            words_in_query.add(word)

        for d, words_in_doc in self.docs.items():
            scores[d] = len(words_in_query.intersection(words_in_doc)) \
                        / float(len(words_in_query.union(words_in_doc)))

        # ------------------------------------------------------------------

        ranking = [idx for idx, sim in sorted(enumerate(scores),
                                              key=lambda xx: xx[1],
                                              reverse=True)]
        results = []
        for i in range(10):
            results.append((ranking[i], scores[ranking[i]]))
        return results

Let's also add a few methods that given a string, will process and then return the list of matching documents for the different methods you have implemented. You do not need to add any additional code here.

In [17]:
class IRSystem(IRSystem): 
    def process_query(self, query_str):
        """
        Given a query string, process it and return the list of lowercase,
        alphanumeric, stemmed words in the string.
        """
        # make sure everything is lower case
        query = query_str.lower()
        # split on whitespace
        query = query.split()
        # remove non alphanumeric characters
        query = [self.alphanum.sub('', xx) for xx in query]
        # stem words
        query = [self.p.stem(xx) for xx in query]
        return query

    def query_retrieve(self, query_str):
        """
        Given a string, process and then return the list of matching documents
        found by boolean_retrieve().
        """
        query = self.process_query(query_str)
        return self.boolean_retrieve(query)

    def phrase_query_retrieve(self, query_str):
        """
        Given a string, process and then return the list of matching documents
        found by phrase_retrieve().
        """
        query = self.process_query(query_str)
        return self.phrase_retrieve(query)

    def query_rank(self, query_str):
        """
        Given a string, process and then return the list of the top matching
        documents, rank-ordered.
        """
        query = self.process_query(query_str)
        return self.rank_retrieve(query)

In [95]:
set(1)

TypeError: 'int' object is not iterable

## Running the code
You can use the run_query function to test your code on a specific query. 

Note that the first time to run the run_query function, it will create a directory named stemmed/ in ../data/RiderHaggard/. This is meant to be a simple cache for the raw text documents. Later runs will be much faster after the first run. However, this means that if something happens during this first run and it does not get through processing all the documents, you may be left with an incomplete set of documents in ../data/RiderHaggard/stemmed/. If this happens, simply remove the stemmed/ directory and re-run!

In [18]:
def run_tests(irsys):
    print("===== Running tests =====")

    ff = open('./data/dev_queries.txt')
    questions = [xx.strip() for xx in ff.readlines()]
    ff.close()
    ff = open('./data/dev_solutions.txt')
    solutions = [xx.strip() for xx in ff.readlines()]
    ff.close()

    epsilon = 1e-4
    for part in range(6):
        points = 0
        num_correct = 0
        num_total = 0

        prob = questions[part]
        soln = json.loads(solutions[part])

        if part == 0:  # inverted index test
            print("Inverted Index Test")
            queries = prob.split("; ")
            queries = [xx.split(", ") for xx in queries]
            queries = [(xx[0], int(xx[1])) for xx in queries]
            for i, (word, doc) in enumerate(queries):
                num_total += 1
                guess = irsys.get_word_positions(word, doc)
                if sorted(guess) == soln[i]:
                    num_correct += 1

        if part == 1:  # get postings test
            print("Get Postings Test")
            words = prob.split(", ")
            for i, word in enumerate(words):
                num_total += 1
                posting = irsys.get_posting_unstemmed(word)
                if posting == soln[i]:
                    num_correct += 1

        elif part == 2:  # boolean retrieval test
            print("Boolean Retrieval Test")
            queries = prob.split(", ")
            for i, query in enumerate(queries):
                num_total += 1
                guess = irsys.query_retrieve(query)
                if set(guess) == set(soln[i]):
                    num_correct += 1

        elif part == 3:  # phrase query test
            print("Phrase Query Retrieval")
            queries = prob.split(", ")
            for i, query in enumerate(queries):
                num_total += 1
                guess = irsys.phrase_query_retrieve(query)
                if set(guess) == set(soln[i]):
                    num_correct += 1

        elif part == 4:  # tfidf test
            print("TF-IDF Test")
            queries = prob.split("; ")
            queries = [xx.split(", ") for xx in queries]
            queries = [(xx[0], int(xx[1])) for xx in queries]
            for i, (word, doc) in enumerate(queries):
                num_total += 1
                guess = irsys.get_tfidf_unstemmed(word, doc)
                if guess >= float(soln[i]) - epsilon and \
                        guess <= float(soln[i]) + epsilon:
                    num_correct += 1

        elif part == 5:  # cosine similarity test
            print("Cosine Similarity Test")
            queries = prob.split(", ")
            for i, query in enumerate(queries):
                num_total += 1
                ranked = irsys.query_rank(query)
                top_rank = ranked[0]
                if top_rank[0] == soln[i][0]:
                    if top_rank[1] >= float(soln[i][1]) - epsilon and \
                            top_rank[1] <= float(soln[i][1]) + epsilon:
                        num_correct += 1

        feedback = "%d/%d Correct. Accuracy: %f" % \
                   (num_correct, num_total, float(num_correct) / num_total)

        if part == 1:
            if num_correct == num_total:
                points = 2
            elif num_correct >= 0.5 * num_total:
                points = 1
            else:
                points = 0
        elif part == 2:
            if num_correct == num_total:
                points = 1
            else:
                points = 0
        else:
            if num_correct == num_total:
                points = 3
            elif num_correct > 0.75 * num_total:
                points = 2
            elif num_correct > 0:
                points = 1
            else:
                points = 0

        print("    Score: %d Feedback: %s" % (points, feedback))

In [19]:
## Run this cell to run the tests
irsys = IRSystem()
irsys.read_data('./data/RiderHaggard')
irsys.index()
irsys.compute_tfidf()

run_tests(irsys)

Reading in documents...
Already stemmed!
Indexing...
Calculating tf-idf...
===== Running tests =====
Inverted Index Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
Get Postings Test
    Score: 2 Feedback: 5/5 Correct. Accuracy: 1.000000
Boolean Retrieval Test
    Score: 1 Feedback: 5/5 Correct. Accuracy: 1.000000
Phrase Query Retrieval
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
TF-IDF Test
    Score: 0 Feedback: 0/5 Correct. Accuracy: 0.000000
Cosine Similarity Test
    Score: 0 Feedback: 0/5 Correct. Accuracy: 0.000000


In [176]:
irsys_debug = IRSystem()
irsys_debug.read_data('./data/RiderHaggardDebug')
irsys_debug.index()
irsys_debug.compute_tfidf()

Reading in documents...
Already stemmed!
filenames
Indexing...
['titl', 'a', 'winter', 'pilgrimag', 'author', 'h', 'rider', 'haggard', 'first', 'publish', '1901', 'be', 'an', 'account', 'of', 'travel', 'through', 'palestin', 'itali', 'and', 'the', 'island', 'of', 'cypru', 'accomplish', 'in', 'the', 'year', '1900', 'dedic', 'i', 'offer', 'these', 'page', 'to', 'mr', 'mr', 'hart', 'bennett', 'and', 'all', 'other', 'cyprian', 'friend', 'whose', 'hospit', 'and', 'kind', 'have', 'made', 'my', 'sojourn', 'in', 'the', 'island', 'so', 'pleasant', 'to', 'rememb', 'ditchingham', '1901', 'a', 'winter', 'pilgrimag', 'chapter', 'i', 'milan', 'cathedr', 'sure', 'solomon', 'foresaw', 'these', 'dai', 'when', 'he', 'set', 'down', 'that', 'famou', 'sai', 'as', 'to', 'the', 'make', 'of', 'mani', 'book', 'the', 'aphor', 'i', 'confess', 'is', 'on', 'which', 'strike', 'me', 'through', 'with', 'shame', 'whenev', 'i', 'chanc', 'to', 'be', 'call', 'upon', 'to', 'read', 'it', 'aloud', 'in', 'the', 'parish', 'ch

In [177]:
irsys_debug.inv_index

{'titl': {0: [0]},
 'a': {0: [1, 61, 121, 125, 127, 136, 157], 1: [40, 55, 68, 77, 96, 107]},
 'winter': {0: [2, 62]},
 'pilgrimag': {0: [3, 63]},
 'author': {0: [4]},
 'h': {0: [5], 1: [17]},
 'rider': {0: [6], 1: [18]},
 'haggard': {0: [7], 1: [19]},
 'first': {0: [8]},
 'publish': {0: [9]},
 '1901': {0: [10, 60]},
 'be': {0: [11, 103], 1: [51]},
 'an': {0: [12, 144], 1: [12, 93]},
 'account': {0: [13, 212]},
 'of': {0: [14, 22, 84, 141, 154, 159, 162, 180, 217],
  1: [14, 36, 46, 57, 65, 80, 103, 110, 122, 125, 148]},
 'travel': {0: [15, 161]},
 'through': {0: [16, 96]},
 'palestin': {0: [17]},
 'itali': {0: [18]},
 'and': {0: [19, 39, 46, 175, 186, 190, 200], 1: [90, 124, 140]},
 'the': {0: [20, 26, 53, 82, 87, 111, 139, 152, 173, 178, 184, 187, 211],
  1: [9, 34, 47, 58, 73, 146]},
 'island': {0: [21, 54]},
 'cypru': {0: [23]},
 'accomplish': {0: [24]},
 'in': {0: [25, 52, 110, 172, 204, 214], 1: [30, 33, 92]},
 'year': {0: [27]},
 '1900': {0: [28]},
 'dedic': {0: [29]},
 'i': {0:

In [178]:
run_tests(irsys_debug)

===== Running tests =====
Inverted Index Test


KeyError: 'separ'

In [23]:
def run_query(query):
    irsys = IRSystem()
    irsys.read_data('./data/RiderHaggard')
    irsys.index()
    irsys.compute_tfidf()
    
    print("Best matching documents to '%s':" % query)
    results = irsys.query_rank(query)
    for docId, score in results:
        print("%s: %e" % (irsys.titles[docId], score))
        
## Example query run where "My very own query" is your query.
run_query("My very own query")

Reading in documents...
Already stemmed!
Indexing...
Calculating tf-idf...
Best matching documents to 'My very own query':
Long Odds: 2.240478e-03
Hunter Quatermain's Story: 2.103787e-03
The Tale of Three Lions: 1.574803e-03
The Mahatma and the Hare: 1.283697e-03
Black Heart and White Heart: 1.139818e-03
Elissa: 1.117943e-03
Moon of Israel: 1.059883e-03
Morning Star: 9.930487e-04
Maiwa's Revenge: 9.526834e-04
Doctor Therne: 9.261403e-04


## Information Retrieval Systems and Search Engines

Safiya Umoja Noble’s <a href="https://nyupress.org/9781479837243/algorithms-of-oppression/">Algorithms of Oppression</a> (2018) provides insight into how search engines and information retrieval algorithms can exhibit substantial racist and sexist biases. Noble demonstrates how prejudice against black women is embedded into search engine ranked results. These biases are apparent in both Google search’s autosuggestions and the first page of Google results. In this assignment, we have explored numerous features that are built into information retrieval systems, like Google Search. 

How could the algorithms you built in this assignment contribute to enforcing real-world biases? 

How can we reduce data discrimination and algorithmic bias that perpetuate gender and racial inequalities in search results and IR system?

Please provide a short response to each of these questions (1-2 paragraphs per question).

In [ ]:
class IRSystem(IRSystem):
    def algorithmic_bias_in_IR_systems(self):
        # TODO: Place your response into the response string below
        response = ""
        return response

Once you're ready to submit, you can run the cell below to prepare and zip
up your solution:

In [ ]:
%%bash

if [[ ! -f "./pa3.ipynb" ]]
then
    echo "WARNING: Did not find notebook in Jupyter working directory. This probably means you're running on Google Colab. You'll need to go to File->Download .ipynb to download your notebok and other files, then zip them locally. See the README for more information."
else
    echo "Found notebook file, creating submission zip..."
    zip -r submission.zip pa3.ipynb deps/
fi

If you're running on Google Colab, see the README for instructions on
how to submit.

__Best of luck!__

__Some reminders for submission:__
* If you have any extra files required for your implementation to work, make
 sure they are in a `deps/` folder on the same level as `pa3.ipynb` and
 include that folder in your submission zip file.
 * Make sure you didn't accidentally change the name of your notebook file,
 (it should be `pa3.ipynb`) as that is required for the autograder to work.
* Go to Gradescope (gradescope.com), find the PA3 IR assignment and
upload your zip file (`submission.zip`) as your solution.
* Wait for the autograder to run (it should only take a minute or so) and check
that your submission was graded successfully! If the autograder fails, or you
get an unexpected score it may be a sign that your zip file was incorrect.